I now realize hastags may be another interesting add on, like #nbapreseason vs #nbafinals. Clearly one of those is more interesting than the other, and perhaps that would lead to a different engagment rate. 


In an exploratory study I'll find the most common hashtags, and plot their engagement rates. 

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('husl')
%matplotlib inline

/Users/noahkasmanoff/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
instas = pd.read_csv('Business Analytics/training_set.csv',encoding = 'unicode_escape')

In [44]:
def get_postscount(z,driver):
    """
    
    Obtains total number of posts to use a given hashtag, found in hashtag df. 
    
    Parameters
    ----------
    
    z : str
        The hashtag name, done via an apply statement. 
        
    driver : WebDriver
        Selenium webdriver object, used to naviagate to the URL of each account, and then grab the corresponding
        followers. 
        
    Returns 
    -------
    
    nposts : float
        The number of posts of that particular hashtag. 

    """
    from time import sleep
    from random import randint
    print("Hashtag: ", z)

    z = z.replace('#','')#.replace(' ','').replace('Â','').replace('å','').replace('â','') #clean unnecesary characters
   # print("Account: ", z)
    url = "https://www.instagram.com/explore/tags/" + z +"/?hl=en"
    driver.get(url)
    sleep(randint(1,3))
    nposts = driver.find_elements_by_class_name('g47SY')[0].text #total number of posts to use this hastag. 
    print("# of posts with this hashtag = " , nposts)
    return nposts


def load_hashtags(instas):
    """
    
    Creates a dataset summarizing the instagram hashtags used in the post. 
    The columns used are a temporary list, and this line will be finalized once it is. 
    
    
    Parameters
    ----------
    
    instas : df
        Dataframe of the instagram posts, their associated information + output of engagements
        
        
    Returns
    -------
    
    
    hastag_df : df
    
        Dataframe of the hashtagss tagged in any or multiple posts. 
        
    """
    #use regular expression to obtain any time a name is used, find all
    import re

    posts = " ".join(instas['Description'].unique()).lower()
    hashtags = list( dict.fromkeys(re.findall('#[a-z_0-9]*',posts)) ) 
    instas['Description'] = instas['Description'].str.lower()

   # print("Total profiles: ", len(tagged_profiles))
    #iterate through these accounts, obtain info regarding how they are engaged. 
    d = []
    for hashtag in hashtags:
        posts_w_hashtag = instas.loc[instas['Description'].str.contains(hashtag)]
        max_eng = posts_w_hashtag.Engagements.max()
        tot_eng = posts_w_hashtag.Engagements.sum()
        n_posts = posts_w_hashtag.Engagements.count()
        min_eng = posts_w_hashtag.Engagements.min()
        avg_eng = posts_w_hashtag.Engagements.mean()
        var_eng = posts_w_hashtag.Engagements.std()
        d.append({'hashtags': hashtag, 'max_eng': max_eng,'tot_eng' : tot_eng, 
                 'n_posts' : n_posts, 'min_eng' : min_eng, 'avg_eng' : avg_eng,  'var_eng' : var_eng})

    hashtags_df = pd.DataFrame(d)
    #but we're not done, can also establish how popular these accounts are, doing it based on follower count. 
    from selenium import webdriver
    driver = webdriver.Chrome('/Users/noahkasmanoff/Desktop/chromedriver')

    hashtag_df['# of posts']  = hashtag_df['hashtags'].apply(lambda z: get_postscount(z,driver))
    driver.close()
    
    
    return hashtags_df

In [45]:
instas = pd.read_csv('Business Analytics/training_set.csv',encoding = 'unicode_escape')

instas.dropna(inplace=True)


hashtag_df = load_hashtags(instas.head(n=1))

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Hashtag:  #nbafinals
# of posts with this hashtag =  1,644,786
Hashtag:  #gatoradezero


KeyboardInterrupt: 

In [43]:
instas.head(n=1)['Description'].values[0]

'the @raptors bench trio of @sergeibaka @normanpowell4 & @fredvanvleet combine for 48 in game 4!'

In [32]:
hashtag_df.shape

(349, 7)

In [33]:
hashtag_df.drop_duplicates(subset=['hashtags']).shape

(349, 7)

In [34]:
hashtag_df.dropna()

,avg_eng,hashtags,max_eng,min_eng,n_posts,tot_eng,var_eng
0,561512.662420,#nbafinals,909585,213824,157,88157488,180787.279867
2,517760.528814,#nbaplayoffs,954448,198217,590,305478712,167667.393603
3,363438.486486,#tripledoublealert,729066,281503,111,40341672,61314.782017
4,590180.000000,#powerbeatspro,775600,404760,2,1180360,262223.478735
5,563701.000000,#legendarymoments,833726,357280,13,7328113,122344.292696
6,618668.694805,#nbabreakdown,890873,412240,154,95274979,90474.182868
7,438375.567568,#nbaawards,807274,251554,37,16219896,161934.649229
8,624522.750000,#nbacombine,690297,471492,8,4996182,70440.511257
9,569828.666667,#nbaplaymakers,757785,471492,3,1709486,162833.301116
10,645666.672269,#phantomcam,838340,383348,119,76834334,85964.653242
